In [11]:
# fetch pdb bpti and write to file

import sys
import os
import pdbtools
import subprocess
pdb_code = "5PTI"

conditions = ("APO", "amber99")

suffix = "_".join(conditions)

pdbfile = pdb_code + ".pdb"

outname = pdb_code + "_" + suffix + ".pdb"


In [2]:
#TODO download and use amber14 forcefield
# we need to make adjustments for deuterium
# for now just rename to hydrogens 

In [3]:
!pdb_fetch $pdb_code > $pdbfile



In [4]:
!pdb_delhetatm $pdbfile  > $outname


In [5]:
# convert D to H using mdanalysis
import MDAnalysis as mda

h_name = outname.replace(".pdb", "_H.pdb")

u = mda.Universe(outname)

for atom in u.atoms:
    if atom.element == "D" or atom.type == "D":
        name = atom.name
        atom.name = name.replace("D", "H")
        atom.element = "H"
        atom.type = "H"

u.atoms.write(h_name)



/Users/alexi/miniconda/envs/top_gen/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/alexi/miniconda/envs/top_gen/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib
/Users/alexi/miniconda/envs/top_gen/lib/python3.11/site-packages/MDAnalysis/topology/PDBParser.py:348: UserWarning: Unknown element D found for some atoms. These have been given an empty element record. If needed they can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(wmsg)
/Users/alexi/miniconda/envs/top_gen/lib/python3.11/site-packages/MDAnalysis/topology/guessers.py:146: UserWarning: Failed to guess the mass for the following atom types: D
  warnings.warn("Failed to guess the mass for the following atom ty

In [6]:
ph = 7.0
temp = 300.0

#pdb2gmx for now - update with tleap later

pdb2gmx_cmd = ["gmx","pdb2gmx", 
               "-f", h_name, 
               "-o", h_name.replace(".pdb", ".gro"), 
               "-p", h_name.replace(".pdb", ".top"),
               "-water", "spc", 
            #    "-ff", "amber99sb-ildn.ff", 
               "-ignh"]

print(" ".join(pdb2gmx_cmd))

gmx pdb2gmx -f 5PTI_APO_amber99_H.pdb -o 5PTI_APO_amber99_H.gro -p 5PTI_APO_amber99_H.top -water spc -ignh


In [7]:
# place in box 
editconf_cmd = ["gmx", "editconf", 
                "-f", h_name.replace(".pdb", ".gro"), 
                "-o", h_name.replace(".pdb", "_box.gro"), 
                "-c", "-d", "1.0", "-bt", "cubic"]

print(" ".join(editconf_cmd))
subprocess.run(editconf_cmd )

gmx editconf -f 5PTI_APO_amber99_H.gro -o 5PTI_APO_amber99_H_box.gro -c -d 1.0 -bt cubic


In [8]:
# and solvate and add ions
solvate_cmd = ["gmx", "solvate", 
               "-cp", h_name.replace(".pdb", "_box.gro"),
               "-cs", "spc216.gro", 
               "-o", h_name.replace(".pdb", "_solv.gro"), 
               "-p", h_name.replace(".pdb", ".top")]

print(" ".join(solvate_cmd))
subprocess.run(pdb2gmx_cmd)

gmx solvate -cp 5PTI_APO_amber99_H_box.gro -cs spc216.gro -o 5PTI_APO_amber99_H_solv.gro -p 5PTI_APO_amber99_H.top


In [15]:
!touch test.mdp
# add ions
grompp_cmd = ["gmx", "grompp", 
              "-f", "test.mdp", 
              "-c", h_name.replace(".pdb", "_solv.gro"), 
              "-p", h_name.replace(".pdb", ".top"), 
              "-o", "ions.tpr"]

print(" ".join(grompp_cmd))
subprocess.run(grompp_cmd)


gmx grompp -f test.mdp -c 5PTI_APO_amber99_H_solv.gro -p 5PTI_APO_amber99_H.top -o ions.tpr
Setting the LD random seed to -4194319

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein_chain_A'

Excluding 2 bonded neighbours molecule type 'SOL'
Analysing residue names:
There are:    58    Protein residues
There are:  6547      Water residues
Analysing Protein...

This run will generate roughly 2 Mb of data
gmx genion -s ions.tpr -o 5PTI_APO_amber99_H_solv_ions.gro -p 5PTI_APO_amber99_H.top -pname NA -nname CL -neutral -conc 0.15


                      :-) GROMACS - gmx grompp, 2023.3 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /Users/alexi/Library/CloudStorage/OneDrive-Nexus365/Rotation_Projects/Rotation_3/Project/topology_generation
Command line:
  gmx grompp -f test.mdp -c 5PTI_APO_amber99_H_solv.gro -p 5PTI_APO_amber99_H.top -o ions.tpr


NOTE 1 [file test.mdp]:
  For a correct single-point energy evaluation with nsteps = 0, use
  continuation = yes to avoid constraining the input coordinates.

Generating 1-4 interactions: fudge = 0.5

NOTE 2 [file 5PTI_APO_amber99_H.top, line 8555]:
  System has non-zero total charge: 6.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




NOTE 3 [file 5PTI_APO_amber99_H.top, line 8555]:
  The bond in molecule-type Protein_chain_A between atoms 158 OH and 159 HH
  has an estimated oscillational per

In [16]:

genion_cmd = ["gmx", "genion",
                "-s", "ions.tpr", 
                "-o", h_name.replace(".pdb", "_solv_ions.gro"), 
                "-p", h_name.replace(".pdb", ".top"), 
                "-pname", "NA", "-nname", "CL", 
                "-neutral", "-conc", "0.15"]
print(" ".join(genion_cmd))

gmx genion -s ions.tpr -o 5PTI_APO_amber99_H_solv_ions.gro -p 5PTI_APO_amber99_H.top -pname NA -nname CL -neutral -conc 0.15
